<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3>Part I: Prepare Data Frame</h3>

In this part we will download table from Wikipedia and prepare data frame with Boroughs data.   
    To get data we will use pandas and read_html function  
    Rename columns to be inline with Coursera Submission Instruction  
    Remove all "Not assigned" Boroughs   

In [8]:
import pandas as pd
read_table=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
temp_df=read_table[0]
temp_df.rename(columns={'Postal Code': 'PostalCode', 'Borough':'Borough', 'Neighbourhood': 'Neighborhood'}, inplace=True)
toronto_df=temp_df[temp_df['Borough']!='Not assigned'].reset_index(drop=True)
toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
toronto_df.shape

(103, 3)

<h3>Part II: Add Longitude and Latitude to Toronto Data</h3>

In this part, using csv file, we will add two columns to our data set, one with Longitude and another one with Latitude.

In [16]:
import numpy as np
latlon_df=pd.read_csv('http://cocl.us/Geospatial_data')
latlon_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
columns_name=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
final_set=[]
for index, singleRow in toronto_df.iterrows():
    final_set.append([singleRow['PostalCode'], singleRow['Borough'], singleRow['Neighborhood'], latlon_df[latlon_df['Postal Code']==singleRow['PostalCode']]['Latitude'].values[0], latlon_df[latlon_df['Postal Code']==singleRow['PostalCode']]['Longitude'].values[0]])
final_df=pd.DataFrame(final_set, columns=columns_name)
final_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [27]:
final_df.shape

(103, 5)